# RD performance of the 2D-DWT

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import math
import numpy as np
from scipy import signal
import cv2
import os
import pywt
import pylab
!ln -sf ~/MRVC/src/deadzone.py .
import deadzone
!ln -sf ~/MRVC/src/frame.py .
import frame
!ln -sf ~/MRVC/src/YCoCg.py .
import YCoCg
!ln -sf ~/MRVC/src/DWT.py .
import DWT

In [ ]:
def q_deq(x, quantization_step):
    k = deadzone.quantize(x, quantization_step)
    y = deadzone.dequantize(k, quantization_step)
    return k, y

In [ ]:
def read_indexes(prefix):
    frame.read(prefix)
    
def write_indexes(prefix):
    frame.save(prefix)

In [ ]:
fn = "/home/vruiz/MRVC/sequences/stockholm_5_frames/000"
RGB_frame = frame.read(fn)

In [ ]:
def show(img, title):
    img = frame.normalize(img)
    plt.figure(figsize=(10,10))
    plt.title(title, fontsize=20)
    plt.imshow(img)

In [ ]:
show(RGB_frame, "stockholm000")

In [ ]:
def average_energy(x):
    return np.sum(x.astype(np.double)*x.astype(np.double))/(np.size(x))

def MSE(x, y):
    error_signal = x.astype(np.float32) - y
    return average_energy(error_signal)

def RMSE(x, y):
    error_signal = x.astype(np.float32) - y
    return math.sqrt(MSE(error_signal))

In [ ]:
def bytes_per_frame(_frame):
    frame.write(_frame, "/tmp/frame")
    length_in_bytes = os.path.getsize("/tmp/frame.png")
    return length_in_bytes

def bytes_per_grayframe(_frame):
    cv2.imwrite("/tmp/frame.png", _frame)
    length_in_bytes = os.path.getsize("/tmp/frame.png")
    return length_in_bytes

## Quantizing the RGB domain

In [ ]:
def only_Q_RD_curve(frame):
    points = []
    for q_step in range(0, 8):
        k, y = q_deq(frame, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bytes_per_frame(k)
        distortion = MSE(frame, y)
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
        points.append((rate, distortion))
    return points

RGB_points = only_Q_RD_curve(RGB_frame)

## Quantizing the YCoCg domain

In [ ]:
def YCoCg_RD_curve(RGB_frame):
    RD_points = []
    for q_step in range(0, 8):
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        k, dequantized_YCoCg_frame = q_deq(YCoCg_frame, 1<<q_step)
        k = k.astype(np.uint8)
        rate = bytes_per_frame(k)
        dequantized_RGB_frame = YCoCg.to_RGB(dequantized_YCoCg_frame)
        distortion = MSE(RGB_frame, dequantized_RGB_frame)
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
        RD_points.append((rate, distortion))
    return RD_points

YCoCg_points = YCoCg_RD_curve(RGB_frame)

In [ ]:
WAVELET = pywt.Wavelet("db5")
#WAVELET = pywt.Wavelet("bior3.5")
N_LEVELS = 3

def color_DWT_analyze(color_frame, wavelet=WAVELET, n_levels=N_LEVELS):
    n_channels = color_frame.shape[2]
    color_decomposition = [None]*n_channels
    for c in range(n_channels):
        color_decomposition[c] = pywt.wavedec2(data=color_frame[:,:,c], wavelet=wavelet, mode='per', level=n_levels)
    return color_decomposition # A list of "gray" decompositions

def _color_DWT_analyze(color_frame, wavelet=WAVELET, n_levels=N_LEVELS):
    n_channels = color_frame.shape[0]
    color_decomposition = [None]*n_channels
    for c in range(n_channels):
        color_decomposition[c] = pywt.wavedec2(data=color_frame[c], wavelet=wavelet, mode='per', level=n_levels)
    return color_decomposition # A list of "gray" decompositions

def color_DWT_synthesize(color_decomposition, wavelet=WAVELET):
    n_channels = len(color_decomposition)
    #n_levels = len(color_decomposition[0])-1
    # color_decomposition[0] <- First channel
    # color_decomposition[0][0] <- cAn (lowest frequecy subband) of the first channel
    # color_decomposition[0][1] <- (cHn, cVn, cDn) (lowest high-frequency subbands) of the first channel
    # color_decomposition[0][1][0] <- cHn (LH subband) of the first channel
    # See https://pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#d-multilevel-decomposition-using-wavedec2
    _color_frame = []
    for c in range(n_channels):
        frame = pywt.waverec2(color_decomposition[c], wavelet=wavelet, mode='per')
        _color_frame.append(frame)
    n_rows = _color_frame[0].shape[0]
    n_columns = _color_frame[0].shape[1]
    color_frame = np.ndarray((n_rows, n_columns, n_channels), np.float64)
    for c in range(n_channels):
        color_frame[:,:,c] = _color_frame[c][:,:]
    return color_frame

def _color_DWT_synthesize(color_decomposition, wavelet=WAVELET):
    n_channels = len(color_decomposition)
    #n_levels = len(color_decomposition[0])-1
    # color_decomposition[0] <- First channel
    # color_decomposition[0][0] <- cAn (lowest frequecy subband) of the first channel
    # color_decomposition[0][1] <- (cHn, cVn, cDn) (lowest high-frequency subbands) of the first channel
    # color_decomposition[0][1][0] <- cHn (LH subband) of the first channel
    # See https://pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#d-multilevel-decomposition-using-wavedec2
    _color_frame = []
    for c in range(n_channels):
        _frame = pywt.waverec2(color_decomposition[c], wavelet=wavelet, mode='per')
        _color_frame.append(_frame)
    #n_rows = _color_frame[1].shape[0]
    #n_columns = _color_frame[2].shape[1]
    #color_frame = np.ndarray((n_channels, n_rows, n_columns), np.float64)
    #for c in range(n_channels):
    #    color_frame[c] = _color_frame[c][:,:]
    return np.array(_color_frame)

### Testing reversebility

In [ ]:
RGB_decomposition = color_DWT_analyze(RGB_frame, n_levels=2)
RGB_reconstructed_frame = color_DWT_synthesize(RGB_decomposition)
(RGB_frame == RGB_reconstructed_frame).all()

In [ ]:
RGB_decomposition = DWT.analyze(RGB_frame, n_levels=2)
RGB_reconstructed_frame = DWT.synthesize(RGB_decomposition)
(RGB_frame == RGB_reconstructed_frame).all()

In [ ]:
RGB_reconstructed_frame.shape

In [ ]:
show(RGB_frame, "original")

In [ ]:
show(RGB_reconstructed_frame, "reconstruction")

In [ ]:
show(RGB_frame - RGB_reconstructed_frame, "difference")

In [ ]:
YCoCg_frame = YCoCg.from_RGB(RGB_frame)
YCoCg_decomposition = color_DWT_analyze(YCoCg_frame, n_levels=2)
YCoCg_reconstructed_frame = color_DWT_synthesize(YCoCg_decomposition)
RGB_reconstructed_frame = YCoCg.to_RGB(YCoCg_reconstructed_frame)
(RGB_frame == RGB_reconstructed_frame).all()

In [ ]:
YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
YCoCg_decomposition = DWT.analyze(YCoCg_frame, n_levels=2)
YCoCg_reconstructed_frame = DWT.synthesize(YCoCg_decomposition)
RGB_reconstructed_frame = YCoCg.to_RGB(YCoCg_reconstructed_frame)
(RGB_frame == RGB_reconstructed_frame).all()

In [ ]:
show(RGB_frame, "original") 

In [ ]:
show(RGB_reconstructed_frame, "reconstruction")

In [ ]:
show(RGB_frame - RGB_reconstructed_frame, "difference")

## Quantizing the YCoCg/DWT domain

In [ ]:
def DWT_RD_curve(RGB_frame, n_levels=N_LEVELS):
    n_channels = RGB_frame.shape[2]
    RD_points = []
    for q_step in range(0, 8):
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        YCoCg_decomposition = DWT.analyze(YCoCg_frame, n_levels=n_levels)
        dequantized_YCoCg_decomposition = []
        rate = 0
        for channel in range(n_channels):
            # In a channel there is a decomposition
            decomposition = YCoCg_decomposition[channel]
            cAn = decomposition[0]
            k, dequantized_cAn = q_deq(cAn, 1<<q_step)
            k = k.astype(np.uint8)
            dequantized_decomposition = [dequantized_cAn]
            rate += bytes_per_frame(k)
            rest_of_resolutions = decomposition[1:]
            for resolution in rest_of_resolutions:
                # In a resolution there is/are one/three subbands
                dequantized_resolution = []
                for subband in resolution:
                    k, dequantized_subband = q_deq(subband, 1<<q_step)
                    k = k.astype(np.uint8)
                    rate += bytes_per_grayframe(k)
                    dequantized_resolution.append(dequantized_subband)
                dequantized_decomposition.append(tuple(dequantized_resolution))
            dequantized_YCoCg_decomposition.append(dequantized_decomposition)
        reconstructed_YCoCg_frame = DWT.synthesize(dequantized_YCoCg_decomposition)
        reconstructed_RGB_frame = YCoCg.to_RGB(reconstructed_YCoCg_frame)
        distortion = MSE(RGB_frame, reconstructed_RGB_frame)
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
        RD_points.append((rate, distortion))
    return RD_points

DWT_points_1 = DWT_RD_curve(RGB_frame, n_levels=1)
DWT_points_2 = DWT_RD_curve(RGB_frame, n_levels=2)
DWT_points_3 = DWT_RD_curve(RGB_frame, n_levels=3)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DWT_points_1), c='r', marker="x", label='DWT(n_levels=1)')
pylab.plot(*zip(*DWT_points_2), c='g', marker="x", label='DWT(n_levels=2)')
pylab.plot(*zip(*DWT_points_3), c='b', marker="x", label='DWT(n_levels=3)')
pylab.title("Impact of the Number of Levels")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RGB_points), c='b', marker="x", label='RGB')
pylab.plot(*zip(*YCoCg_points), c='g', marker="x", label='YCoCg')
pylab.plot(*zip(*DWT_points_3), c='r', marker="x", label='YCoCg+DWT')
pylab.title("Global Performance")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
#plt.yscale('log')
pylab.show()

## Ignore the rest

In [ ]:
def RGB_to_YCoCg(RGB_frame):
    R, G, B = RGB_frame[:,:,0], RGB_frame[:,:,1], RGB_frame[:,:,2]
    YCoCg_frame = np.empty_like(RGB_frame)
    YCoCg_frame[:,:,0] =  R/4 + G/2 + B/4 
    YCoCg_frame[:,:,1] =  R/2       - B/2
    YCoCg_frame[:,:,2] = -R/4 + G/2 - B/4
    return YCoCg_frame

def YCoCg_to_RGB(YCoCg_frame):
    Y, Co, Cg = YCoCg_frame[:,:,0], YCoCg_frame[:,:,1], YCoCg_frame[:,:,2]
    RGB_frame = np.empty_like(YCoCg_frame)
    RGB_frame[:,:,0] = Y + Co - Cg 
    RGB_frame[:,:,1] = Y      + Cg
    RGB_frame[:,:,2] = Y - Co - Cg
    return RGB_frame